## Full Convolutional Neural Network with hyperparameter optimization

We're going to train a full, regularized CNN architecture with some automatic hyperperameter optimization using `hyperas` (a wrapper for `hyperopt`).

In [1]:
import datetime
import os
import logging
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import tools.train as train
import tools.models as models

## Read in data
files = ("../data/mitbih_train.csv", "../data/mitbih_test.csv")
inputs, labels, sparse_labels, df = train.preprocess(*files, fft=False)
# Add a dimension for "channels"
for key in inputs:
    inputs[key] = tf.expand_dims(inputs[key], axis=2)
train.class_count(df)


Train set
Count of each class
Classes
0.0    72471
1.0     2223
2.0     5788
3.0      641
4.0     6431
dtype: int64

Test set
Count of each class
Classes
0.0    18118
1.0      556
2.0     1448
3.0      162
4.0     1608
dtype: int64


In [ ]:
# Suppress tensorflow warnings about internal deprecations
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# functions for hyperas
def data():
    return inputs["train"], labels["train"], inputs["test"], labels["test"]

def create_model(x_train, y_train, x_test, y_test):
    
    nblocks = {{choice([4, 8])}}
    nfilters = 32
    dilation_limit = inputs["train"].shape[1]
    
    # Start
    layerlist_res = [("conv", {"filters": nfilters, "width": 1, "padding": "causal"})]

    # Residual blocks
    models.add_res_blocks(nblocks, nfilters, dilation_limit, layerlist_res)

    # End
    layerlist_res.extend([
        (layers.Activation("relu"),),
        ("conv", {"filters": nfilters, "width": 1, "padding": "causal"}),
        ("conv", {"filters": 1, "width": 1, "padding": "causal"}),
        (layers.Dropout({{uniform(0, 1)}},),)
    ])

    config = {"optimizer": {{choice(["Nadam", "sgd"])}},
              "loss": "categorical_crossentropy",
              "batch_size": {{choice([50, 100, 200])}},
              "val_split": 0.05,
              "epochs": 300,
              "verbose": 0,
              "patience": 20}

    inputsize = inputs["train"].shape[1]
    ncategories = labels["train"].shape[1]
    model_res = models.create_conv1d(inputsize, layerlist_res, ncategories)
    history = train.train(model_res, inputs, labels, config)
    
    # get the highest validation accuracy of the training epochs
    validation_acc = np.amax(history.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model_res}

best_run, best_model = optim.minimize(
    model=create_model,
    data=data,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

print("Chosen hyperparameters from the best-trained model")
print(best_run)

print(
    "Train acc of best performing model:",
    model.evaluate(inputs["train"], labels["train"], verbose=0)[1],
)
print(
    "Test acc of best performing model:",
    model.evaluate(inputs["test"], labels["test"], verbose=0)[1],

test_pred = np.argmax(model_res.predict(inputs["test"]), axis=1)
plot.plot_cm(sparse_labels["test"], test_pred, classes=["N", "S", "V", "F", "Q"], normalize=True)